In [67]:
import pandas as pd
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
pd.set_option('display.width', 1000)
pd.set_option('display.max_colwidth', 500)
from langchain.schema import Document
from sentence_transformers import SentenceTransformer
import faiss
import numpy as np
import os
from pprint import pprint
from langchain_community.vectorstores import FAISS
# from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_huggingface import HuggingFaceEmbeddings
from langchain.schema import Document
import os
import duckdb



In [ ]:
!pip install langchain datasets transformers langchain_community langchain_openai langchain_text_splitters faiss-cpu sentence-transformers

     ---------------------------------------- 0.0/1.0 MB ? eta -:--:--
     -- ------------------------------------- 0.1/1.0 MB 4.1 MB/s eta 0:00:01
     ----- ---------------------------------- 0.1/1.0 MB 1.7 MB/s eta 0:00:01
     ---------- ----------------------------- 0.3/1.0 MB 2.3 MB/s eta 0:00:01
     ---------- ----------------------------- 0.3/1.0 MB 1.9 MB/s eta 0:00:01
     -------------- ------------------------- 0.4/1.0 MB 1.8 MB/s eta 0:00:01
     -------------------- ------------------- 0.5/1.0 MB 2.0 MB/s eta 0:00:01
     ----------------------- ---------------- 0.6/1.0 MB 2.0 MB/s eta 0:00:01
     ---------------------------- ----------- 0.7/1.0 MB 2.1 MB/s eta 0:00:01
     ------------------------------- -------- 0.8/1.0 MB 1.9 MB/s eta 0:00:01
     ----------------------------------- ---- 0.9/1.0 MB 1.9 MB/s eta 0:00:01
     ---------------------------------------  1.0/1.0 MB 1.9 MB/s eta 0:00:01
     ---------------------------------------- 1.0/1.0 MB 1.9 MB/s eta 0


[notice] A new release of pip is available: 23.0.1 -> 25.1.1
[notice] To update, run: python.exe -m pip install --upgrade pip


# Explore data

In [ ]:

info = pd.read_csv("../data/info_particulier_impot.csv", sep = ",")
question =pd.read_csv("../data/questions_fiches_fip.csv", sep = ",")

In [51]:
info[info["Unnamed: 0"]==44]

,Unnamed: 0,niveau_0,niveau_1,URL,Titre,Texte
44,44,https://www.impots.gouv.fr/particulier/payer-mes-impots-taxes,https://www.impots.gouv.fr/particulier/je-dois-payer-une-amende-ou-un-forfait-de-post-stationnement,https://www.impots.gouv.fr/je-paie-mon-amende-ou-mon-forfait-de-post-stationnement-0,JE PAIE MON AMENDE OU MON FORFAIT DE POST-STATIONNEMENT,Je paie mon amende ou mon forfait de post-stationnement\n1\nJe paie mon amende\nJe paie mon forfait de post-stationnement\n2\nJe paie par internet (sur mon ordinateur ou mon smartphone)\nJe paie chez un buraliste agréé (paiement de proximité)\nJe paie par téléphone\nJe paie par chèque\nJe paie auprès d'un centre des Finances publiques\nJe suis un redevable étranger et je paye mon amende forfaitaire majorée radar\nJe paie par internet (sur mon ordinateur ou mon smartphone)\nJe paie chez un bu...


In [50]:
question[question["Unnamed: 0"]==44]

,Unnamed: 0,question,num_texte
670,44,Dans quel cas un contribuable peut-il cocher la case G ?,3


In [30]:
info.head(1).to_dict(orient="records")

[{'Unnamed: 0': 0,
  'niveau_0': 'https://www.impots.gouv.fr/particulier/declarer-mes-revenus',
  'niveau_1': 'https://www.impots.gouv.fr/particulier/je-declare-pour-la-premiere-fois-je-declare-chaque-annee',
  'URL': 'https://www.impots.gouv.fr/particulier/lobligation-de-declarer-vos-revenus-chaque-annee',
  'Titre': 'OBLIGATIONS DÉCLARATIVES',
  'Texte': "Obligations déclaratives\nL'obligation de déclarer vos revenus chaque année\nSi vous êtes dans l’une des situations suivantes, vous devez souscrire une déclaration des revenus :\nvous avez en France votre foyer (résidence habituelle) ou votre lieu de séjour principal (en règle générale si vous y séjournez pendant plus de 6 mois par an)\xa0;\nvous exercez votre activité professionnelle principale en France\xa0;\nvous avez en France le centre de vos intérêts économiques (vos principaux investissements ou le siège de vos affaires sont en France).\nSi vous avez eu 18 ans l'année d'imposition des revenus et que vous n'êtes plus rattaché 

# RAGS from Scracth
**Objectif** : Développer, from scratch, une architecture RAG en 4 modules: 

- Lecture et découpage du csv (une ligne = un document)
- Générer les embeddings
- Récupérer les passages pertinents en fonction d'une requête

In [84]:
def load_csv_to_duckdb(csv_path: str, db_path: str, table_name: str = "info_particulier_impot"):
    df = pd.read_csv(csv_path)

    con = duckdb.connect(database=db_path)
    con.execute(f"DROP TABLE IF EXISTS {table_name}")
    con.execute(f"CREATE TABLE {table_name} AS SELECT * FROM df")
    con.close()
    print(f"✅ Données chargées dans DuckDB : {db_path} [{table_name}]")


In [85]:
load_csv_to_duckdb(
        csv_path="../data/info_particulier_impot.csv",
        db_path="../data/data.duckdb"
    )

✅ Données chargées dans DuckDB : ../data/data.duckdb [info_particulier_impot]


In [ ]:

def load_documents_from_csv(csv_path: str) -> list[Document]:
    df = pd.read_csv(csv_path, sep=",")

    documents = []
    for _, row in df.iterrows():
        titre = row.get("Titre", "")
        texte = row.get("Texte", "")
        url = row.get("URL", "")
        content = f"{titre}\n\n{texte}"

        metadata = {
            "source": url,
            "titre": titre
        }

        documents.append(Document(page_content=content, metadata=metadata))

    return documents


In [87]:

def load_documents_from_duckdb(db_path: str, table_name: str = "info_particulier_impot") -> list[Document]:
    con = duckdb.connect(database=db_path)
    df = con.execute(f"SELECT * FROM {table_name}").fetchdf()
    con.close()

    documents = []
    for _, row in df.iterrows():
        titre = row.get("Titre", "")
        texte = row.get("Texte", "")
        url = row.get("URL", "")
        content = f"{titre}\n\n{texte}"

        metadata = {
            "source": url,
            "titre": titre
        }

        documents.append(Document(page_content=content, metadata=metadata))

    return documents



In [92]:

def inspect_documents(documents, n=3):
    for i, doc in enumerate(documents[:n], 1):
        print(f"\n--- Chunk {i} ---")
        print("Titre :", doc.metadata.get("titre", ""))
        print("Source :", doc.metadata.get("source", ""))
        print("Contenu :")
        print(doc.page_content[:500])  # Limite d'affichage pour pas tout inonder
        print("-" * 60)

inspect_documents(
                  load_documents_from_csv("../data/info_particulier_impot.csv")
)


--- Chunk 1 ---
Titre : OBLIGATIONS DÉCLARATIVES
Source : https://www.impots.gouv.fr/particulier/lobligation-de-declarer-vos-revenus-chaque-annee
Contenu :
OBLIGATIONS DÉCLARATIVES

Obligations déclaratives
L'obligation de déclarer vos revenus chaque année
Si vous êtes dans l’une des situations suivantes, vous devez souscrire une déclaration des revenus :
vous avez en France votre foyer (résidence habituelle) ou votre lieu de séjour principal (en règle générale si vous y séjournez pendant plus de 6 mois par an) ;
vous exercez votre activité professionnelle principale en France ;
vous avez en France le centre de vos intérêts économiques (vos princip
------------------------------------------------------------

--- Chunk 2 ---
Titre : MODALITÉS DE DÉCLARATION
Source : https://www.impots.gouv.fr/particulier/modalites-de-declaration
Contenu :
MODALITÉS DE DÉCLARATION

Modalités de déclaration
Le principe : une seule déclaration de revenus par foyer fiscal
Pour les personnes seules (céliba

In [91]:
inspect_documents(load_documents_from_duckdb("../data/data.duckdb"))
    



--- Chunk 1 ---
Titre : OBLIGATIONS DÉCLARATIVES
Source : https://www.impots.gouv.fr/particulier/lobligation-de-declarer-vos-revenus-chaque-annee
Contenu :
OBLIGATIONS DÉCLARATIVES

Obligations déclaratives
L'obligation de déclarer vos revenus chaque année
Si vous êtes dans l’une des situations suivantes, vous devez souscrire une déclaration des revenus :
vous avez en France votre foyer (résidence habituelle) ou votre lieu de séjour principal (en règle générale si vous y séjournez pendant plus de 6 mois par an) ;
vous exercez votre activité professionnelle principale en France ;
vous avez en France le centre de vos intérêts économiques (vos princip
------------------------------------------------------------

--- Chunk 2 ---
Titre : MODALITÉS DE DÉCLARATION
Source : https://www.impots.gouv.fr/particulier/modalites-de-declaration
Contenu :
MODALITÉS DE DÉCLARATION

Modalités de déclaration
Le principe : une seule déclaration de revenus par foyer fiscal
Pour les personnes seules (céliba

In [93]:
# Module 2 : Génération des embeddings et stockage avec FAISS

def create_faiss_index(documents: list[Document], save_path: str = "faiss_index") -> FAISS:
    # Embeddings via Hugging Face
    embedding_model = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )

    # Création de l'index FAISS
    faiss_index = FAISS.from_documents(documents, embedding_model)

    # Sauvegarde locale
    faiss_index.save_local(save_path)

    return faiss_index


In [100]:
docs = load_documents_from_csv("../data/info_particulier_impot.csv")
index = create_faiss_index(docs, save_path="faiss_data2")

# Pour requêter
retrieved_docs = index.similarity_search("Qui doit souscrire une déclaration de revenus chaque année ?", k=3)

for doc in retrieved_docs:
    print(doc.metadata["titre"])
    print(doc.metadata["source"])
    print(doc.page_content[:300], "\n---")


COMMENT DÉCLARER VOS REVENUS ?
https://www.impots.gouv.fr/particulier/comment-declarer-votre-impot
COMMENT DÉCLARER VOS REVENUS ?

Comment déclarer vos revenus ?
Vous quittez la France
En cas de départ à l’étranger, la première chose à faire est de signaler votre changement d'adresse auprès du Service des Impôts des Particuliers (SIP) duquel vous dépendez. Ce signalement est important afin que vo 
---
VOTRE CONJOINT(E) EST DÉCÉDÉ(E)
https://www.impots.gouv.fr/particulier/votre-conjointe-est-decedee
VOTRE CONJOINT(E) EST DÉCÉDÉ(E)

Votre conjoint(e) est décédé(e)
Deux déclarations de revenus au titre de l'année du décès
L’année du décès de votre conjoint ou partenaire de Pacs (N), vous avez deux déclarations de revenus à souscrire en ligne dès lors que vous étiez auparavant soumis à une imposit 
---
L'AVIS D'IMPÔT SUR LES REVENUS
https://www.impots.gouv.fr/particulier/lavis-dimpot-sur-les-revenus
L'AVIS D'IMPÔT SUR LES REVENUS

L'avis d'impôt sur les revenus
À quoi correspond cet avis ?

In [ ]:
# Module 3 : Recherche des passages pertinents avec une requête

def load_faiss_index(load_path: str = "faiss_data2") -> FAISS:
    embedding_model = HuggingFaceEmbeddings(
        model_name="sentence-transformers/all-MiniLM-L6-v2"
    )
    return FAISS.load_local(
        load_path, 
        embedding_model, 
        allow_dangerous_deserialization=True  # à activer uniquement pour des documents vérifiées
    )

In [ ]:
index = load_faiss_index("faiss_data2")

# Requête utilisateur
query = "case G"
results = index.similarity_search(query, k=3)

for doc in results:
    print(doc.metadata["titre"])
    print(doc.metadata["source"])
    print(doc.page_content[:300], "\n---")

JE SOUHAITE CONTESTER MON AMENDE OU MON FORFAIT DE POST-STATIONNEMENT
https://www.impots.gouv.fr/je-souhaite-contester-mon-amende-ou-mon-forfait-de-post-stationnement-0
JE SOUHAITE CONTESTER MON AMENDE OU MON FORFAIT DE POST-STATIONNEMENT

Je souhaite contester mon amende ou mon forfait de post-stationnement
1
Je conteste une amende radar (vitesse, feu-rouge etc.)
Je conteste toute autre nature d’amende
Je souhaite contester mon forfait de post-stationnement
2
Je c 
---
LES REVENUS MOBILIERS
https://www.impots.gouv.fr/particulier/les-revenus-mobiliers
LES REVENUS MOBILIERS

Les revenus mobiliers
Les revenus imposables
Vos revenus mobiliers proviennent des valeurs mobilières que vous possédez (actions, parts de SARL, obligations, bons de capitalisation, contrats d'assurance-vie, etc.). Ils ne doivent pas être confondus avec les plus-values mobiliè 
---
J’AI UNE QUESTION SUR MON AMENDE OU MON FORFAIT DE POST-STATIONNEMENT
https://www.impots.gouv.fr/jai-une-question-sur-mon-amende-ou-mon-

In [103]:
results

[Document(id='f245e8ac-9c9e-458e-ac6f-51b7fdb7d1cf', metadata={'source': 'https://www.impots.gouv.fr/je-souhaite-contester-mon-amende-ou-mon-forfait-de-post-stationnement-0', 'titre': 'JE SOUHAITE CONTESTER MON AMENDE OU MON FORFAIT DE POST-STATIONNEMENT'}, page_content="JE SOUHAITE CONTESTER MON AMENDE OU MON FORFAIT DE POST-STATIONNEMENT\n\nJe souhaite contester mon amende ou mon forfait de post-stationnement\n1\nJe conteste une amende radar (vitesse, feu-rouge etc.)\nJe conteste toute autre nature d’amende\nJe souhaite contester mon forfait de post-stationnement\n2\nJe conteste une amende forfaitaire radar (vitesse, feu-rouge etc.)\nJe conteste une amende forfaitaire majorée radar (vitesse, feu-rouge etc.)\nJe conteste une amende forfaitaire d’une autre nature que radar\nJe conteste une amende forfaitaire délictuelle ou une amende forfaitaire majorée délictuelle\nJe conteste une amende forfaitaire majorée SNCF ou RATP\nJe conteste un avis d'amende faisant suite à une condamnation pr

In [105]:

# Génère une réponse en utilisant l’API du LLM local + le contexte FAISS
def generate(user_input, previous_context):
    index = load_faiss_index()
    retrieved_context = get_context_from_query(index, user_input)

    prompt = f"""Voici des documents utiles : 

{retrieved_context}

En te basant sur ces informations, réponds à la question suivante : 
{user_input}
"""

    r = requests.post(
        "http://127.0.0.1:11434/api/generate",
        json={
            "model": model,
            "prompt": prompt,
            "context": previous_context,
        },
        stream=True,
    )
    r.raise_for_status()

    for line in r.iter_lines():
        body = json.loads(line)
        response_part = body.get("response", "")
        print(response_part, end="", flush=True)

        if "error" in body:
            raise Exception(body["error"])

        if body.get("done", False):
            return body["context"]
